**qAIntum.ai**

# **Quantum Neural Network (QNN) Classifier**

This note is an example of using Photonic Analog (PA) QNN for binary classification. This is an application of the original work ["Continuous Variable Quantum Neural Networks"](https://arxiv.org/abs/1806.06871).

Compared to Classical Neural Networks, PA QNNs have a reduced number of parameters to train and converge faster with fewer epochs. However, this is a quantum algorithm simulated on classical computers hence the training time for quantum circuits tend to be longer than classical models.

The dataset used in this example can be found at https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

This file is organized in the following order:
0. Install and import necessary packages
1. Load and preprocess data (The data from Kaggle is saved as 'financial.csv'.)
2. Data encoding
3. QNN model
   * QNN layer
   * QNN circuit
   * Model building
4. Model training
5. Evaluation

For the open source repository, refer to https://github.com/qaintumai/quantum.

## **0. Intall Packages**

Ensure you have installed the packages detailed in requirements.txt before running the code below.

[Pennylane](https://pennylane.ai/) is a Python based quantum machine learning library by Xanadu. An old version is necessary for this example: v0.29.1.

In [133]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml
from sklearn.preprocessing import StandardScaler

## **1. Load and Preprocess Data**

Data: 

Data points: 

Number of features: 

Label: 

For the purpose of this experiment, we are using only ___ data points for training and __ for testing.

data source: 

In [136]:
#LOADING DATA
import sys
import os
script_dir = os.getcwd()
data_path = os.path.join(script_dir, '../examples/data/diabetes.csv')
data_path = os.path.normpath(data_path)

# Load the dataset
dataset = np.loadtxt(data_path, delimiter=',')

print("length of dataset: ", len(dataset))

X = dataset[:,0:8]
y = dataset[:,8]

# Apply StandardScaler for normalization
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Convert the normalized data to torch tensors
X = torch.tensor(X_normalized, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

print(f"Size of X tensor: {X.size()} and first element of X: {X[0]}")
print(f"Size of Y tensor: {y.size()} and first element of Y: {y[0]}")

length of dataset:  768
Size of X tensor: torch.Size([768, 8]) and first element of X: tensor([ 0.6399,  0.8483,  0.1496,  0.9073, -0.6929,  0.2040,  0.4685,  1.4260])
Size of Y tensor: torch.Size([768, 1]) and first element of Y: tensor([1.])


## **2. Data Encoding**

This step converts classical data into a quantum state by using the data entries as parameters of the quantum gates.

The data encoding gates used are Squeezing, Rotation, Beamsplitter, Displacement Gate, and Kerr Gate. Other gates under "CV operators" in the Pennylane package can be explored.

In [19]:
def encode(x, num_wires):
        """
        Encodes the input data into a quantum state to be operated on using a sequence of quantum gates.

        Parameters:
        x : input data (list or array-like)

        The encoding process uses the following gates in sequence:
        - Squeezing gates: 2*self.num_wires parameters
        - Beamsplitter gates: 2(self.num_wires-1) parameters
        - Rotation gates: self.num_wires parameters
        - Displacement gates: 2*self.num_wires parameters
        - Kerr gates: self.num_wires parameters
          Total: 8*self.num_wires - 2 parameters

        rounds: the number of iterations of the sequence needed to take in all the entries of the input data
                num_features // (8 * self.num_wires - 2)
                We are adding (8 * self.num_wires - 3) as a pad to run one extra round for the remainding data entries.
        """
        num_features = len(x)
        

        # Calculate the number of rounds needed to process all features
        rounds = (num_features + (8 * num_wires - 3)) // (8 * num_wires - 2)
        print("encoding rounds", rounds)

        for j in range(rounds):
            start_idx = j * (8 * num_wires - 2)

            # Squeezing gates
            for i in range(num_wires):
                # for each wire, the number of parameters are i*2
                idx = start_idx + i * 2
                if idx + 1 < num_features:
                    qml.Squeezing(x[idx], x[idx + 1], wires=i)

            # Beamsplitter gates
            for i in range(num_wires - 1):
                # start_index + Squeezing gates, and then i*2 parameters for each gate
                idx = start_idx + num_wires * 2 + i * 2
                if idx + 1 < num_features:
                    qml.Beamsplitter(x[idx], x[idx + 1], wires=[i % num_wires, (i + 1) % num_wires])

            # Rotation gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters, and then i parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + i
                if idx < num_features:
                    qml.Rotation(x[idx], wires=i)

            # Displacement gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters + Rotation gates, and then i*2 parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + num_wires + i * 2
                if idx + 1 < num_features:
                    qml.Displacement(x[idx], x[idx + 1], wires=i)

            # Kerr gates
            for i in range(num_wires):
                # start_index + Squeezing gates + Beamsplitters + Rotation gates + Displacement gates, and then i parameters for each gate
                idx = start_idx + num_wires * 2 + (num_wires - 1) * 2 + num_wires + num_wires * 2 + i
                if idx < num_features:
                    qml.Kerr(x[idx], wires=i)

## **3. QNN Model**
To build a model, we need to
* define a layer
* build a circuit with the defined layer
* build a model with the defined circuit.

### **3.1 QNN Layer**
This in a faithful implementation of classical neural networks:
* Weight matrix: Interferometer 1 + Squeezing + Interferometer 2
* Bias addition: Displacement gate
* Nonlinear activation function: Kerr gate

In [20]:
def apply(v, num_wires):
        """
        Applies the quantum neural network layer with the given parameters.

        Parameters:
        - v (list or array): List or array of parameters for the quantum gates.

        Returns:
        - None
        """
        num_params = len(v)

        # Interferometer 1
        for i in range(num_wires - 1):
            idx = i * 2
            if idx + 1 < num_params:
                theta = v[idx]
                phi = v[idx + 1]
                qml.Beamsplitter(theta, phi, wires=[i % num_wires, (i + 1) % num_wires])

        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + i
            if idx < num_params:
                qml.Rotation(v[idx], wires=i)

        # Squeezers
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + i
            if idx < num_params:
                qml.Squeezing(v[idx], 0.0, wires=i)

        # Interferometer 2
        for i in range(num_wires - 1):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + i * 2
            if idx + 1 < num_params:
                theta = v[idx]
                phi = v[idx + 1]
                qml.Beamsplitter(theta, phi, wires=[i % num_wires, (i + 1) % num_wires])

        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + i
            if idx < num_params:
                qml.Rotation(v[idx], wires=i)

        # Bias addition
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + num_wires + i
            if idx < num_params:
                qml.Displacement(v[idx], 0.0, wires=i)

        # Non-linear activation function
        for i in range(num_wires):
            idx = (num_wires - 1) * 2 + num_wires + num_wires + (num_wires - 1) * 2 + num_wires + num_wires + i
            if idx < num_params:
                qml.Kerr(v[idx], wires=i)


**Weight Initializer**

Randomly initialized values are used as initial parameters of the QNN circuit.

In [120]:
def init_weights(layers, num_wires, active_sd=0.0001, passive_sd=0.1):
    """
    This is a weight vector initializer.
    Input: number of layers, number of wires
    Output: concatenated weight vector
    """
    M = (num_wires - 1) * 2 + num_wires  # Number of interferometer parameters

    int1_weights = np.random.normal(size=[layers, M], scale=passive_sd) #beamsplitters and rotations
    print("size for int1", int1_weights.size)
    s_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #squeezers
    print("size for s_weights", int1_weights.size)
    int2_weights = np.random.normal(size=[layers, M], scale=passive_sd) #beamsplitters and rotations
    print("size for int2", int2_weights.size)
    dr_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #displacement
    print("size for dr_weights", dr_weights.size)
    k_weights = np.random.normal(size=[layers, num_wires], scale=active_sd) #Kerr
    print("size for k_weights", k_weights.size)

    weights = np.concatenate(
        [int1_weights, s_weights, int2_weights, dr_weights, k_weights], axis=1)
    
    return weights

### **3.2 QNN Circuit**

For building a PA circuit as opposed to a qubit-based circuit, we have to choose "strawberryfields.fock" as device.

In [121]:
num_wires = 4
num_basis = 2
# Global variables

encoding = True
batching = False  # Set this to True if you want to encode multiple times (e.g., for batching)

def encode_data_once(x, num_wires):
    """ Encodes the data only once and stores it. """
    global encoded_data
    encode(encoded_data, num_wires)
    print(f"ENCODED inputs: {encoded_data}")

@qml.qnode(dev, interface="torch")
def quantum_nn(inputs, var):
    """ Quantum Neural Network with a caching mechanism for encoded inputs. """
    global encoding
    if batching:  # Encode every time if batching is true
        encode(inputs, num_wires)
        print(f"ENCODED inputs: {inputs}")
    elif encoding:  # Encode only once if batching is false
        encode_data_once(inputs, num_wires)
        encoding = False

    print("length of var", len(var))
    print("shape of var", var.shape)
    for v in var:
        print("--- Applying Layer ---")
        print(f"first element in v: {v[0]}")
        apply(v, num_wires)
    return qml.expval(qml.X(0))

### **3.3 Model building**

In [138]:
num_layers = 2

def get_model(num_wires, num_layers):
    """
    This is a model building function.
    Input: number of modes, number of layers
    Output: PyTorch model
    """
    weights = init_weights(num_layers, num_wires)
    print("shape of weights", weights.shape)
    print("weights length: ", len(weights))
    print("weights looks like: ", weights[0][1])

    shape_tup = weights.shape
    weight_shapes = {'var': shape_tup}
    qlayer = qml.qnn.TorchLayer(quantum_nn, weight_shapes)
    model = torch.nn.Sequential(
        qlayer,          # Quantum layer
#         nn.Sigmoid()     # Sigmoid activation
    )
    return model

In [139]:
model = get_model(num_wires, num_layers)
print(model.parameters())
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of model parameters: {total_params}")
for p in model.parameters():
    print("parameter: ", p)

size for int1 20
size for s_weights 20
size for int2 20
size for dr_weights 8
size for k_weights 8
shape of weights (2, 32)
weights length:  2
weights looks like:  0.0699847594924983
<generator object Module.parameters at 0x187d68740>
Sequential(
  (0): <Quantum Torch Layer: func=quantum_nn>
)
Total number of model parameters: 64
parameter:  Parameter containing:
tensor([[2.5591, 4.6573, 2.0116, 1.8993, 4.4306, 0.6941, 2.4440, 5.6482, 0.4854,
         4.9190, 0.0154, 0.1668, 0.7276, 2.5447, 0.9796, 1.1051, 5.8074, 1.6498,
         1.1792, 0.7413, 2.8697, 2.0903, 0.4754, 6.1403, 2.3994, 1.9473, 5.7063,
         3.9678, 5.0263, 4.4565, 4.6194, 2.2535],
        [5.0341, 1.1076, 0.6948, 3.5792, 0.9459, 4.8303, 2.5926, 5.1890, 3.2412,
         4.2835, 5.9311, 3.1942, 3.9092, 3.5606, 6.0353, 0.1464, 1.7154, 1.5911,
         4.0821, 5.3627, 0.4276, 4.1319, 1.5163, 1.9070, 5.3828, 0.0313, 3.7916,
         4.2988, 3.5729, 0.8238, 3.8386, 3.5388]], requires_grad=True)


## **4. Model Training**

In [ ]:
# TRAINING MODEL (without batching)
loss_fn = torch.nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

n_epochs = 3

for epoch in range(n_epochs):
    print("====================================================================================")
    print(f"+++++++++++++++ Epoch {epoch} ++++++++++++++++++++++++++++++++++")
    print("====================================================================================")

    # Forward pass: compute predicted y by passing X to the model
    y_pred_prob = model(X).reshape(-1, 1)  
    print(f"Size of y_pred_prob: {y_pred_prob.size()}, Values: {y_pred_prob}")

    # Calculate loss using the predicted probabilities (before thresholding)
    loss = loss_fn(y_pred_prob, y)
    print(f"loss = {loss}")

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f'Finished epoch {epoch}, latest loss {loss}')


# #TRAINING MODEL
# loss_fn = nn.BCELoss()  # binary cross entropy
# optimizer = optim.Adam(model.parameters(), lr=0.01)

# n_epochs = 1
# batch_size = 2
 
# for epoch in range(n_epochs):
#     for i in range(0, len(X), batch_size):
#         print("====================================================================================")
#         print(f"+++++++++++++++Batch number: {i} in Epoch {epoch}++++++++++++++++++++++++++++++++++")
#         print("====================================================================================")

#         Xbatch = X[i:i+batch_size]
#         print(f"Size of Xbatch: {Xbatch.size()}")
#         y_pred = model(Xbatch).reshape(-1, 1)  
#         print(f"Size of y_pred: {y_pred.size()}, Values: {y_pred}")

#         ybatch = y[i:i+batch_size]
#         loss = loss_fn(y_pred, ybatch)
#         print(f"loss = {loss}")
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
    
#     print(f'Finished epoch {epoch}, latest loss {loss}')


+++++++++++++++ Epoch 0 ++++++++++++++++++++++++++++++++++
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element 

length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first elemen

length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first elemen

length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first elemen

length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first elemen

...

loss = 34.89583206176758
Finished epoch 1, latest loss 34.89583206176758
+++++++++++++++ Epoch 2 ++++++++++++++++++++++++++++++++++
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var

length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first elemen

length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first elemen

length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first elemen

length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first element in v: 5.034058570861816
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
first element in v: 2.559128761291504
--- Applying Layer ---
first elemen

## **5. Evaluation**

In [137]:
# EVALUATINGwith torch.no_grad():
    y_pred_prob = model(X)  # Get the probabilities from the model
    y_pred = (y_pred_prob >= 0.5).float()  # Convert probabilities to binary predictions

    print(f"Predicted probabilities: {y_pred_prob}")
    print(f"Binary predictions: {y_pred}")

    # Calculate accuracy or other metrics
    accuracy = (y_pred == y).float().mean()
    print(f"Accuracy: {accuracy}")

IndentationError: unexpected indent (3348920376.py, line 2)

## **ISOLATED TESTING**

Beware and steer clear

In [102]:
# Global variables
encoded_data = None
encoding = True
batching = False  # Set this to True if you want to encode multiple times (e.g., for batching)

def encode_data_once(x, num_wires):
    """ Encodes the data only once and stores it. """
    global encoded_data
    if encoded_data is None:
        encoded_data = x.clone()  # Store the original data before encoding
        encode(encoded_data, num_wires)
        print(f"ENCODED inputs: {encoded_data}")
    else:
        print("Using cached encoded inputs.")

@qml.qnode(dev, interface="torch")
def quantum_nn(inputs, var):
    """ Quantum Neural Network with a caching mechanism for encoded inputs. """
    global encoding
    if batching:  # Encode every time if batching is true
        encode(inputs, num_wires)
        print(f"ENCODED inputs: {inputs}")
    elif encoding:  # Encode only once if batching is false
        encode_data_once(inputs, num_wires)
        encoding = False

    print("length of var", len(var))
    print("shape of var", var.shape)
    for v in var:
        print("--- Applying Layer ---")
        print(f"Parameters for this layer: {v}")
        apply(v, num_wires)
    return qml.expval(qml.X(0))

In [103]:
# Simple test input
test_input = torch.tensor([0, 0.5, 0.2, 0.3, 0.4, 0.6, 0.9], dtype=torch.float32)

# Initialize weights with known values or use random values for testing
test_weights = init_weights(num_layers, num_wires, active_sd=0.01, passive_sd=0.1)
test_weights = torch.tensor(test_weights, dtype=torch.float32)

# Define a simple model with the quantum layer
model = get_model(num_wires, num_layers)

# Set up the loss function and optimizer
loss_fn = nn.L1Loss()  # Mean Absolute Error Loss
optimizer = optim.SGD(model.parameters(), lr=0.5)

# Target output for the test case (arbitrary for testing purposes)
y_test = torch.tensor([1.0], dtype=torch.float32)

# Run the training loop for 5 iterations
n_epochs = 5

for epoch in range(n_epochs):
    print("====================================================================================")
    print(f"+++++++++++++++ Epoch {epoch} ++++++++++++++++++++++++++++++++++")
    print("====================================================================================")

    # Forward pass: predict using the model
    y_pred = model(test_input).reshape(-1, 1)
    print(f"Output from the quantum circuit with test input: {y_pred.item()}")

    # Compute loss
    loss = loss_fn(y_pred, y_test)
    print(f"Loss = {loss.item()}")

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Finished epoch {epoch}, latest loss {loss.item()}')

    
#scalar input
#sigmoid output
#add threshold

size for int1 20
size for s_weights 20
size for int2 20
size for dr_weights 8
size for k_weights 8
size for int1 20
size for s_weights 20
size for int2 20
size for dr_weights 8
size for k_weights 8
shape of weights (2, 32)
weights length:  2
weights looks like:  0.15828907991990507
+++++++++++++++ Epoch 0 ++++++++++++++++++++++++++++++++++
encoding rounds 1
ENCODED inputs: tensor([0.0000, 0.5000, 0.2000, 0.3000, 0.4000, 0.6000, 0.9000])
length of var 2
shape of var torch.Size([2, 32])
--- Applying Layer ---
Parameters for this layer: tensor([4.2419, 5.1476, 0.9820, 1.4381, 2.8150, 2.1164, 5.4272, 4.5027, 0.7415,
        6.1218, 5.4651, 1.6242, 0.4775, 3.8790, 4.5786, 3.6405, 3.4437, 5.1381,
        4.8305, 1.1667, 2.8453, 2.9520, 2.5254, 5.6679, 2.3600, 6.1306, 6.2205,
        2.4925, 1.5602, 2.9234, 3.8476, 3.1174], grad_fn=<UnbindBackward0>)
--- Applying Layer ---
Parameters for this layer: tensor([0.9960, 3.3690, 3.5808, 5.6723, 4.4831, 1.6799, 3.4978, 4.6811, 2.0470,
        4.7125